In [1]:
data_var = '2024-02-06'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11310,2024-02-06,Brasil Nbb,19:30,Unifacisa,Fortaleza B.C.,1.36,2.97,162.5,1.85,1.85,-8.5,2.09,1.63,UJfVkMMj,0.735294,0.336700,0.540541,0.540541,0.071994,0.6,0.6,NaN,NaN,199.998,134.845877,0.674236,220.550,126.227800,0.572332,149.246,272.234,118.44,235.84,0.0,0.0,0.0,0.0,0.525839,0.000000,0.174876,2.92,0.584,0.616438,0.731215,1.0,0.268785,-5.00,-1.000,-1.970000,0.343864,0.3,-0.043864
11311,2024-02-06,Brasil Nbb,20:00,Botafogo,Flamengo,7.06,1.08,161.5,1.82,1.88,13.5,2.03,1.67,6yvEE5ai,0.141643,0.925926,0.549451,0.531915,0.067569,0.4,0.8,NaN,NaN,210.604,107.060749,0.508351,108.910,17.348821,0.159295,178.252,108.326,113.96,101.92,0.0,0.0,0.0,0.0,1.038943,0.022933,0.137599,-2.91,-0.582,-10.412371,0.425313,0.3,-0.125313,-1.51,-0.302,-0.264901,0.793396,0.7,-0.093396
11312,2024-02-06,Brasil Nbb,20:00,Corinthians Paulista,Bauru,1.70,2.07,157.5,1.82,1.88,-3.5,2.00,1.69,pSxIDPEc,0.588235,0.483092,0.549451,0.531915,0.071327,0.6,0.6,NaN,NaN,124.952,40.896210,0.327295,177.356,38.902597,0.219348,137.314,171.722,89.06,182.16,0.0,0.0,0.0,0.0,0.138795,0.022933,0.118809,-2.64,-0.528,-1.325758,0.657502,0.5,-0.157502,4.13,0.826,1.295400,0.477021,0.4,-0.077021
11313,2024-02-06,Eua Nba,21:00,Indiana Pacers,Houston Rockets,1.39,2.69,239.5,1.97,1.97,-5.5,1.83,1.83,8xCgM774,0.719424,0.371747,0.507614,0.507614,0.091172,0.4,0.6,NaN,NaN,282.314,132.419597,0.469051,231.644,40.018406,0.172758,255.312,454.478,136.88,269.10,0.0,0.0,0.0,0.0,0.450607,0.000000,0.000000,3.00,0.600,0.650000,0.539105,0.7,0.160895,-3.55,-0.710,-2.380282,0.393796,0.3,-0.093796
11314,2024-02-06,Eua Nba,21:30,Miami Heat,Orlando Magic,1.54,2.26,211.5,1.80,2.02,-4.5,2.08,1.75,G2F1KoxH,0.649351,0.442478,0.555556,0.495050,0.091829,0.2,0.6,NaN,NaN,211.196,66.597146,0.315333,243.260,96.867077,0.398204,223.258,226.936,214.70,140.97,0.0,0.0,0.0,0.0,0.267956,0.081447,0.121851,-3.07,-0.614,-0.879479,0.567532,0.4,-0.167532,0.70,0.140,9.000000,0.432064,0.5,0.067936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11383,2024-02-06,Israel Liga Leumit,14:00,Maccabi Haifa,Elitzor Netania,2.56,1.47,165.5,1.85,1.85,4.5,1.87,1.83,E5Xhea3m,0.390625,0.680272,0.540541,0.540541,0.070897,0.0,0.0,NaN,NaN,211.582,74.838725,0.353710,140.606,48.031746,0.341605,0.000,0.000,120.24,141.95,0.0,0.0,0.0,0.0,0.382504,0.000000,0.015289,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11384,2024-02-06,Israel Liga Leumit,14:15,Maccabi Raanana,Elitzur Shomron,1.30,3.31,170.5,1.85,1.85,-7.5,1.83,1.87,zFYlduls,0.769231,0.302115,0.540541,0.540541,0.071346,0.0,0.0,NaN,NaN,147.044,17.428888,0.118528,183.990,53.657960,0.291635,0.000,0.000,128.80,263.70,0.0,0.0,0.0,0.0,0.616609,0.000000,0.015289,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11385,2024-02-06,Israel Liga Leumit,14:30,Gilboa Galil,Maccabi Maale Adumim,1.31,3.25,156.5,1.81,1.89,-7.5,1.85,1.85,MVz1gwYa,0.763359,0.307692,0.552486,0.529101,0.071051,0.0,0.0,NaN,NaN,156.494,50.161451,0.320533,201.738,93.246012,0.462213,0.000,0.000,164.50,173.84,0.0,0.0,0.0,0.0,0.601661,0.030578,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11386,2024-02-06,Rússia Copa Da Rússia,14:00,Runa Basket Moscow,Nizhny Novgorod,2.05,1.77,147.5,1.82,1.92,-2.5,2.05,1.65,Uibf4IIt,0

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:30,Eua Nba,Brooklyn Nets,Dallas Mavericks,234.5,1.85,1.0,Over
